In [1]:
import numpy as np 
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from category_encoders.binary import BinaryEncoder

from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_trans = pd.read_csv('kaggle_data/credit_card_transactions-ibm_v2.csv').sample(n=1000000, random_state=42)

In [4]:
unique_Users = df_trans['User'].unique()
selected_cards = pd.Series(unique_Users).sample(frac=1.0, replace=False, random_state=42)
filtered_transactions = df_trans[df_trans['User'].isin(selected_cards)]

In [5]:
df = filtered_transactions

In [6]:
df = df[df['Year'].isin([2015,2016])]
len(df)

139998

In [9]:
df["Amount"]=df["Amount"].str.replace("$","").astype(float)
df["Hour"] = df["Time"].str [0:2]
df["Minute"] = df["Time"].str [3:5]
df = df.drop(['Time'],axis=1)
df["Is Fraud?"] = df["Is Fraud?"].apply(lambda x: 1 if x == 'Yes' else 0)


In [10]:
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

# Extract day of the week and map it to its name
days = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
df['Day of Week'] = df['Date'].dt.dayofweek.map(days)


In [11]:
df["card_id"] = df["User"].astype(str) + "_" + df["Card"].astype(str)
df["Merchant"] = df["Merchant Name"].astype(str) + "_" + df["Merchant City"].astype(str)

In [12]:
orig_df = df

In [13]:
columns_to_select = ['Year', 'Day of Week', 'Hour', 'Amount', 'Use Chip', 'Merchant Name', 'MCC', 'Is Fraud?']
df = orig_df[columns_to_select]

In [14]:
orig_df = orig_df.drop(["User","Card"],axis=1)

In [15]:
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

def clean(df):
    # Convert data type
    df['Hour'] = df['Hour'].astype('float')
    
    # Scale the "Amount" column
    scaler = StandardScaler()
    df['Amount'] = scaler.fit_transform(df[['Amount']])
   
    # Binary encoding for categorical variables
    cat_col = ['Use Chip', 'Day of Week']
    for col in cat_col:
        if col in df.columns:
            be = ce.BinaryEncoder(drop_invariant=False)
            enc_df = pd.DataFrame(be.fit_transform(df[col]), dtype='int8')
            df = pd.concat([df, enc_df], axis=1)
            df.drop([col], axis=1, inplace=True)
    
    for col in df.columns:
        df[col] = df[col].astype(float)
        
    return df

# Create the pipeline
preprocessing_pipeline = Pipeline([
    ('cleaning', FunctionTransformer(clean, validate=False)), 
], verbose=True)

df_transformed = preprocessing_pipeline.fit_transform(df)

[Pipeline] .......... (step 1 of 1) Processing cleaning, total=   0.2s


In [16]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and target variable (y)
X = df_transformed.drop(columns=['Is Fraud?'])
y = df_transformed['Is Fraud?']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1613)
train_idx = X_train.index
test_idx = X_test.index
# # Calculate the desired number of fraud cases based on the desired proportion
# desired_proportion = 0.2
# total_samples = 2000
# fraud_samples = int(total_samples * desired_proportion)

# # Create RandomUnderSampler with the desired sampling strategy
# rus = RandomUnderSampler(sampling_strategy={0: total_samples - fraud_samples, 1: fraud_samples}, random_state=1613)

# # Apply random undersampling to the original dataset
# X_resampled, y_resampled = rus.fit_resample(X, y)

# # Split the resampled data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=1613)


In [17]:
orig_df.rename(columns={"Merchant": "merchant_node", "card_id": "client_node", "Is Fraud?":"fraud_label"}, inplace=True)
columns_to_select = ['Year', 'Hour', 'Amount', 'Day of Week', 'Use Chip', 'merchant_node', 'MCC', 'client_node','fraud_label']

graph_df = orig_df[columns_to_select]

In [18]:
graph_df["Day of Week"]=LabelEncoder().fit_transform(graph_df["Day of Week"])
graph_df["Use Chip"]=LabelEncoder().fit_transform(graph_df["Use Chip"])

# graph_df.head()

In [19]:
train_data = graph_df.loc[train_idx]
inductive_data = graph_df.loc[test_idx]

In [20]:
from inductiveGRL.graphconstruction import GraphConstruction

transaction_node_data = train_data.drop("client_node", axis=1).drop("merchant_node", axis=1).drop("fraud_label", axis=1)
client_node_data = pd.DataFrame([1]*len(train_data.client_node.unique())).set_index(train_data.client_node.unique())
merchant_node_data = pd.DataFrame([1]*len(train_data.merchant_node.unique())).set_index(train_data.merchant_node.unique())

nodes = {"client":train_data.client_node, "merchant":train_data.merchant_node, "transaction":train_data.index}
edges = [zip(train_data.client_node, train_data.index),zip(train_data.merchant_node, train_data.index)]
features = {"transaction": transaction_node_data, 'client': client_node_data, 'merchant': merchant_node_data}

graph = GraphConstruction(nodes, edges, features)
S = graph.get_stellargraph()
print(S.info())

StellarGraph: Undirected multigraph
 Nodes: 146912, Edges: 223996

 Node types:
  transaction: [111998]
    Features: float32 vector, length 6
    Edge types: transaction-default->client, transaction-default->merchant
  merchant: [30855]
    Features: float32 vector, length 1
    Edge types: merchant-default->transaction
  client: [4059]
    Features: float32 vector, length 1
    Edge types: client-default->transaction

 Edge types:
    merchant-default->transaction: [111998]
        Weights: all 1 (default)
        Features: none
    client-default->transaction: [111998]
        Weights: all 1 (default)
        Features: none


In [72]:
from inductiveGRL.hinsage import HinSAGE_Representation_Learner

embedding_size = 32
add_additional_data = True
#GraphSAGE parameters
num_samples = [2,64]
embedding_node_type = "transaction"

hinsage = HinSAGE_Representation_Learner(embedding_size, num_samples, embedding_node_type)
trained_hinsage_model, train_emb = hinsage.train_hinsage(S, list(train_data.index), train_data['fraud_label'], batch_size=5, epochs=10)

Epoch 1/10
17920/17920 [==============================] - 59s 3ms/step - loss: 0.0213 - val_loss: 0.0166
Epoch 2/10
17920/17920 [==============================] - 55s 3ms/step - loss: 0.0160 - val_loss: 0.0163
Epoch 3/10
17920/17920 [==============================] - 57s 3ms/step - loss: 0.0159 - val_loss: 0.0162
Epoch 4/10
17920/17920 [==============================] - 56s 3ms/step - loss: 0.0157 - val_loss: 0.0162
Epoch 5/10
17920/17920 [==============================] - 57s 3ms/step - loss: 0.0156 - val_loss: 0.0158
Epoch 6/10
17920/17920 [==============================] - 56s 3ms/step - loss: 0.0155 - val_loss: 0.0158
Epoch 7/10
17920/17920 [==============================] - 57s 3ms/step - loss: 0.0155 - val_loss: 0.0156
Epoch 8/10
17920/17920 [==============================] - 56s 3ms/step - loss: 0.0154 - val_loss: 0.0156
Epoch 9/10
17920/17920 [==============================] - 57s 3ms/step - loss: 0.0153 - val_loss: 0.0154
Epoch 10/10
22400/22400 [==============================

In [ ]:
pd.options.mode.chained_assignment = None

train_data['index'] = train_data.index
inductive_data['index'] = inductive_data.index
inductive_graph_data = pd.concat((train_data,inductive_data))
inductive_graph_data = inductive_graph_data.set_index(inductive_graph_data['index']).drop("index",axis = 1)

transaction_node_data = inductive_graph_data.drop("client_node", axis=1).drop("merchant_node", axis=1).drop("fraud_label", axis=1)
client_node_data = pd.DataFrame([1]*len(inductive_graph_data.client_node.unique())).set_index(inductive_graph_data.client_node.unique())
merchant_node_data = pd.DataFrame([1]*len(inductive_graph_data.merchant_node.unique())).set_index(inductive_graph_data.merchant_node.unique())

nodes = {"client":inductive_graph_data.client_node, "merchant":inductive_graph_data.merchant_node, "transaction":inductive_graph_data.index}
edges = [zip(inductive_graph_data.client_node, inductive_graph_data.index),zip(inductive_graph_data.merchant_node, inductive_graph_data.index)]
features = {"transaction": transaction_node_data, 'client': client_node_data, 'merchant': merchant_node_data}
 
graph = GraphConstruction(nodes, edges, features)
S_combined = graph.get_stellargraph()
print(S_combined.info())

StellarGraph: Undirected multigraph
 Nodes: 178452, Edges: 279996

 Node types:
  transaction: [139998]
    Features: float32 vector, length 6
    Edge types: transaction-default->client, transaction-default->merchant
  merchant: [34386]
    Features: float32 vector, length 1
    Edge types: merchant-default->transaction
  client: [4068]
    Features: float32 vector, length 1
    Edge types: client-default->transaction

 Edge types:
    merchant-default->transaction: [139998]
        Weights: all 1 (default)
        Features: none
    client-default->transaction: [139998]
        Weights: all 1 (default)
        Features: none


In [ ]:
orig_df['client_node'].value_counts()[:40]

In [74]:
from stellargraph.mapper import HinSAGENodeGenerator

test_generator = HinSAGENodeGenerator(
    S_combined,
    batch_size=5,  # Giống batch_size khi train
    num_samples=[2, 64],  # Giống num_samples khi train
    head_node_type="transaction"
).flow(inductive_data.index)  # Chỉ predict trên test data

# 6. Thực hiện dự đoán (FIX lỗi input format)
try:
    # Cách 1: Dùng predict trực tiếp (nếu generator output đúng format)
    test_predictions = trained_hinsage_model.predict(test_generator)
except ValueError:
    # Cách 2: Xử lý thủ công nếu gặp lỗi input format
    test_predictions = []
    for batch in test_generator:
        if isinstance(batch, (list, tuple)):
            batch = batch[0]  # Lấy thành phần chính nếu generator trả về tuple
        preds = trained_hinsage_model.predict(batch)
        test_predictions.extend(preds.squeeze())
    test_predictions = np.array(test_predictions)

1/1 [==============================] - 0s 20ms/step


In [77]:
# df_subset = train_emb.iloc[:, :32]
# train_emb = df_subset       
inductive_emb = pd.DataFrame(np.vstack(test_predictions), index=inductive_data.index)


In [76]:
from sklearn.decomposition import PCA

for i in range(2,11):

    pca = PCA(n_components=i)
    train_emb_pca = pd.DataFrame(pca.fit_transform(train_emb), index=train_emb.index)
    test_emb_pca = pd.DataFrame(pca.transform(inductive_emb), index=inductive_emb.index)


    print(np.sum(pca.explained_variance_ratio_))
    concat_train_emb= pd.merge(train_emb_pca, X_train, left_index=True, right_index=True)
    concat_inductive_emb = pd.merge(test_emb_pca, X_test, left_index=True, right_index=True)
    concat_train_emb.columns = concat_train_emb.columns.astype(str)
    concat_inductive_emb.columns = concat_inductive_emb.columns.astype(str)

    # Tạo mô hình XGBoost
    xgb_classifier = XGBClassifier(n_estimators=100, 
                                use_label_encoder=False,  # tránh warning
                                eval_metric='logloss',    # cần cho phiên bản mới
                                random_state=42)

    # Huấn luyện mô hình
    xgb_classifier.fit(concat_train_emb, y_train)

    # Dự đoán
    y_pred_xgb = xgb_classifier.predict(concat_inductive_emb)

    # Kết quả
    # print("XGBoost Classifier With graph represent Results:")
    # print(classification_report(y_test, y_pred_xgb))
    print(confusion_matrix(y_test, y_pred_xgb))

0.93262788208004
[[27934     1]
 [   18    47]]
0.9896838600295915
[[27934     1]
 [   26    39]]
0.9982095778822037
[[27933     2]
 [   25    40]]
0.9992232646464027
[[27934     1]
 [   25    40]]
0.9997654081795888
[[27935     0]
 [   26    39]]
0.9999286703151121
[[27932     3]
 [   30    35]]
0.9999597725252144
[[27935     0]
 [   30    35]]
0.9999857858184245
[[27933     2]
 [   28    37]]
1.0000074221714634
[[27934     1]
 [   28    37]]


In [ ]:
pca = PCA(n_components=2)
train_emb_pca = pd.DataFrame(pca.fit_transform(train_emb), index=train_emb.index)
test_emb_pca = pd.DataFrame(pca.transform(inductive_emb), index=inductive_emb.index)

concat_train_emb= pd.merge(train_emb_pca, X_train, left_index=True, right_index=True)
concat_inductive_emb = pd.merge(test_emb_pca, X_test, left_index=True, right_index=True)
concat_train_emb.columns = concat_train_emb.columns.astype(str)
concat_inductive_emb.columns = concat_inductive_emb.columns.astype(str)

# Tạo mô hình XGBoost
xgb_classifier = XGBClassifier(n_estimators=100, 
                            use_label_encoder=False,  # tránh warning
                            eval_metric='logloss',    # cần cho phiên bản mới
                            random_state=42)

# Huấn luyện mô hình
xgb_classifier.fit(concat_train_emb, y_train)

# Dự đoán
y_pred_xgb_hinsage = xgb_classifier.predict(concat_inductive_emb)

# Kết quả
# print("XGBoost Classifier With graph represent Results:")
# print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb_hinsage))

[[27934     1]
 [   18    47]]


In [ ]:
xgb_classifier = XGBClassifier(n_estimators=100, 
                            use_label_encoder=False,  # tránh warning
                            eval_metric='logloss',    # cần cho phiên bản mới
                            random_state=42)

# Huấn luyện mô hình
xgb_classifier.fit(X_train, y_train)

# Dự đoán
y_pred_xgb = xgb_classifier.predict(X_test)

# Kết quả
# print("XGBoost Classifier With graph represent Results:")
# print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

[[27933     2]
 [   22    43]]
